## Problem Formulation

The linear programming problem formulation is summarized as below:


<img src="https://drive.google.com/uc?id=1-XmbE04GLWgHa-WHfwCeH4II1tWNKgQ5" style="width: 100px; height=200px"/>


## Setup Environment

In [ ]:
#Install requisite packages
!pip install gurobipy
!pip install cplex
!pip install docplex
!pip install PuLP
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 14.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43.3 MB 1.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 610 kB 13.4 MB/s 
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662847 sha256=59a37da9d2f9c807fd7c2c9571a86ec5cace41daf3eaf8e3ddfe6250db26c0f4
  Stored in directory: /root/.cache/pip/wheels/a7/c9/fb/cee5a89f304e77a39c466e625ac2830434b76eb8384999d116
Successfully built docplex
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 10.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

**Make sure you "Restart Runtime" after installing OR-Tools package for it to be loaded.**

In [ ]:
#Importing Libraries
import numpy as np
import gurobipy as grb
import docplex.mp.model as cpx
import pulp as plp
from scipy.optimize import linprog
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [ ]:
#Data
supply = {'A':70,'B':50,'C':30}
demand = {'D':30,'E':24,'F':12,'G':42,'H':6}
cost_arr = np.array([[16,7,17,14,19],
                    [9,11,16,10,5],
                    [10,18,6,13,8]])

In [ ]:
#Check if cost array is of right shape
cost_arr.shape == (len(supply), len(demand))

True

## Gurobi Implementation

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>1. Initializing Model</h2>
</div>

In [ ]:
gurobi_lp_model = grb.Model()

Restricted license - for non-production use only - expires 2023-10-25


<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>2. Define Decision Variables</h2>
</div>

In [ ]:
quantity_arr  = {(i,j): gurobi_lp_model.addVar(vtype = grb.GRB.INTEGER, name="%s-%s"%(list(supply.keys())[i],list(demand.keys())[j])) for i in range(len(supply)) for j in range(len(demand))}
quantity_arr

{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 4): <gurobi.Var *Awaiting Model Update*>,
 (2, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 4): <gurobi.Var *Awaiting Model Update*>}

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>3. Constraints</h2>
</div>

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(i) Supply Constraint</h4>
</div>

In [ ]:
for i in range(len(supply)):
    gurobi_lp_model.addConstr(sum(quantity_arr[(i,j)] for j in range(len(demand))) <= list(supply.values())[i])
    

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(ii) Demand Constraint</h4>
</div>

In [ ]:
for j in range(len(demand)):
    gurobi_lp_model.addConstr(sum(quantity_arr[(i,j)] for i in range(len(supply))) >= list(demand.values())[j])

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(iii) Non-Negativity Constraint</h4>
</div>

In [ ]:
for i in range(len(supply)):
    for j in range(len(demand)):
        gurobi_lp_model.addConstr(quantity_arr[(i,j)] >= 0)

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>4. Objective Function</h2>
</div>

In [ ]:
gurobi_lp_model.setObjective(sum(cost_arr[i][j]*quantity_arr[(i,j)] for i in range(len(supply)) for j in range(len(demand))), grb.GRB.MINIMIZE)

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>5. Solve Problem</h2>
</div>

In [ ]:
gurobi_lp_model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 23 rows, 15 columns and 45 nonzeros
Model fingerprint: 0x4f2602a0
Variable types: 0 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 7e+01]
Found heuristic solution: objective 1210.0000000
Presolve removed 15 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 15 columns, 30 nonzeros
Variable types: 0 continuous, 15 integer (0 binary)

Root relaxation: objective 1.018000e+03, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1018.0000000 1018.00000  0.00%     -    0s

Explored 1 nodes (7 simplex iterations) in 0.05 

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>6. Results</h2>
</div>

In [ ]:
quantity_arr

{(0, 0): <gurobi.Var A-D (value -0.0)>,
 (0, 1): <gurobi.Var A-E (value 24.0)>,
 (0, 2): <gurobi.Var A-F (value -0.0)>,
 (0, 3): <gurobi.Var A-G (value 10.0)>,
 (0, 4): <gurobi.Var A-H (value -0.0)>,
 (1, 0): <gurobi.Var B-D (value 12.0)>,
 (1, 1): <gurobi.Var B-E (value -0.0)>,
 (1, 2): <gurobi.Var B-F (value -0.0)>,
 (1, 3): <gurobi.Var B-G (value 32.0)>,
 (1, 4): <gurobi.Var B-H (value 6.0)>,
 (2, 0): <gurobi.Var C-D (value 18.0)>,
 (2, 1): <gurobi.Var C-E (value -0.0)>,
 (2, 2): <gurobi.Var C-F (value 12.0)>,
 (2, 3): <gurobi.Var C-G (value -0.0)>,
 (2, 4): <gurobi.Var C-H (value -0.0)>}

In [ ]:
# This prints out the optimal value for all the decision variables
for i in range(len(supply)):
    for j in range(len(demand)):
        print("Optimal shipments from supply city %s to demand city %s is %d"%(list(supply.keys())[i], list(demand.keys())[j], quantity_arr[(i,j)].x))
# This gives the objective value that we were trying to maximize.
print("Total minimum shipping cost is %d"%gurobi_lp_model.objval)

Optimal shipments from supply city A to demand city D is 0
Optimal shipments from supply city A to demand city E is 24
Optimal shipments from supply city A to demand city F is 0
Optimal shipments from supply city A to demand city G is 10
Optimal shipments from supply city A to demand city H is 0
Optimal shipments from supply city B to demand city D is 12
Optimal shipments from supply city B to demand city E is 0
Optimal shipments from supply city B to demand city F is 0
Optimal shipments from supply city B to demand city G is 32
Optimal shipments from supply city B to demand city H is 6
Optimal shipments from supply city C to demand city D is 18
Optimal shipments from supply city C to demand city E is 0
Optimal shipments from supply city C to demand city F is 12
Optimal shipments from supply city C to demand city G is 0
Optimal shipments from supply city C to demand city H is 0
Total minimum shipping cost is 1018


## CPLEX Implementation

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>1. Initializing Model</h2>
</div>

In [ ]:
cplex_lp_model = cpx.Model()

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>2. Define Decision Variables</h2>
</div>

In [ ]:
quantity_arr  = {(i,j): cplex_lp_model.integer_var(name="%s-%s"%(list(supply.keys())[i],list(demand.keys())[j])) for i in range(len(supply)) for j in range(len(demand))}
quantity_arr

{(0, 0): docplex.mp.Var(type=I,name='A-D'),
 (0, 1): docplex.mp.Var(type=I,name='A-E'),
 (0, 2): docplex.mp.Var(type=I,name='A-F'),
 (0, 3): docplex.mp.Var(type=I,name='A-G'),
 (0, 4): docplex.mp.Var(type=I,name='A-H'),
 (1, 0): docplex.mp.Var(type=I,name='B-D'),
 (1, 1): docplex.mp.Var(type=I,name='B-E'),
 (1, 2): docplex.mp.Var(type=I,name='B-F'),
 (1, 3): docplex.mp.Var(type=I,name='B-G'),
 (1, 4): docplex.mp.Var(type=I,name='B-H'),
 (2, 0): docplex.mp.Var(type=I,name='C-D'),
 (2, 1): docplex.mp.Var(type=I,name='C-E'),
 (2, 2): docplex.mp.Var(type=I,name='C-F'),
 (2, 3): docplex.mp.Var(type=I,name='C-G'),
 (2, 4): docplex.mp.Var(type=I,name='C-H')}

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>3. Constraints</h2>
</div>

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(i) Supply Constraint</h4>
</div>

In [ ]:
supply_constraints = {i : cplex_lp_model.add_constraint(ct=cplex_lp_model.sum(quantity_arr[(i,j)] for j in range(len(demand))) <= list(supply.values())[i]) for i in range(len(supply))}

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(ii) Demand Constraint</h4>
</div>

In [ ]:
demand_constraints = {i : cplex_lp_model.add_constraint(ct=cplex_lp_model.sum(quantity_arr[(i,j)] for i in range(len(supply))) >= list(demand.values())[j]) for j in range(len(demand))}

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(iii) Non-Negativity Constraint</h4>
</div>

In [ ]:
non_neg_constraints = {(i,j): cplex_lp_model.add_constraint(ct=quantity_arr[(i,j)] >= 0) for i in range(len(supply)) for j in range(len(demand))}

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>4. Objective Function</h2>
</div>

In [ ]:
objective = cplex_lp_model.sum(cost_arr[i][j]*quantity_arr[(i,j)] for i in range(len(supply)) for j in range(len(demand)))
cplex_lp_model.minimize(objective)

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>5. Solve Problem</h2>
</div>

In [ ]:
cplex_lp_model.solve()

docplex.mp.solution.SolveSolution(obj=1018,values={A-E:24,A-G:10,B-D:12,..

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>6. Results</h2>
</div>

In [ ]:
# This prints out the optimal value for all the decision variables
for i in range(len(supply)):
    for j in range(len(demand)):
        print("Optimal shipments from supply city %s to demand city %s is %d"%(list(supply.keys())[i], list(demand.keys())[j], quantity_arr[(i,j)].solution_value))
# This gives the objective value that we were trying to maximize.
print("Total minimum shipping cost is %d"%cplex_lp_model.objective_value)

Optimal shipments from supply city A to demand city D is 0
Optimal shipments from supply city A to demand city E is 24
Optimal shipments from supply city A to demand city F is 0
Optimal shipments from supply city A to demand city G is 10
Optimal shipments from supply city A to demand city H is 0
Optimal shipments from supply city B to demand city D is 12
Optimal shipments from supply city B to demand city E is 0
Optimal shipments from supply city B to demand city F is 0
Optimal shipments from supply city B to demand city G is 32
Optimal shipments from supply city B to demand city H is 6
Optimal shipments from supply city C to demand city D is 18
Optimal shipments from supply city C to demand city E is 0
Optimal shipments from supply city C to demand city F is 12
Optimal shipments from supply city C to demand city G is 0
Optimal shipments from supply city C to demand city H is 0
Total minimum shipping cost is 1018


## PuLP Implementation

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>1. Initializing Model</h2>
</div>

In [ ]:
pulp_lp_model = plp.LpProblem()

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>2. Define Decision Variables</h2>
</div>

In [ ]:
quantity_arr  = {(i,j): plp.LpVariable(cat=plp.LpInteger, name="%s-%s"%(list(supply.keys())[i],list(demand.keys())[j])) for i in range(len(supply)) for j in range(len(demand))}
quantity_arr

{(0, 0): A_D,
 (0, 1): A_E,
 (0, 2): A_F,
 (0, 3): A_G,
 (0, 4): A_H,
 (1, 0): B_D,
 (1, 1): B_E,
 (1, 2): B_F,
 (1, 3): B_G,
 (1, 4): B_H,
 (2, 0): C_D,
 (2, 1): C_E,
 (2, 2): C_F,
 (2, 3): C_G,
 (2, 4): C_H}

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>3. Constraints</h2>
</div>

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(i) Supply Constraint</h4>
</div>

In [ ]:
supply_constraints = {i : pulp_lp_model.addConstraint(
plp.LpConstraint(
             e=plp.lpSum(quantity_arr[(i,j)] for j in range(len(demand))),
             sense=plp.LpConstraintLE,
             rhs=list(supply.values())[i],
             name="supply_constraint_%s"%(list(supply.keys())[i])))
       for i in range(len(supply))}

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(ii) Demand Constraint</h4>
</div>

In [ ]:
demand_constraints = {j : pulp_lp_model.addConstraint(
plp.LpConstraint(
             e=plp.lpSum(quantity_arr[(i,j)] for i in range(len(supply))),
             sense=plp.LpConstraintGE,
             rhs=list(demand.values())[j],
             name="demand_constraint_%s"%(list(demand.keys())[j])))
       for j in range(len(demand))}

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(iii) Non-Negativity Constraint</h4>
</div>

In [ ]:
non_neg_constraints = {(i,j): pulp_lp_model.addConstraint(
plp.LpConstraint(
             e=quantity_arr[(i,j)],
             sense=plp.LpConstraintGE,
             rhs=0,
             name="non_neg_constraint_{0}_{1}".format(i,j)))
        for i in range(len(supply)) for j in range(len(demand))}

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>4. Objective Function</h2>
</div>

In [ ]:
objective = plp.lpSum(cost_arr[i][j]*quantity_arr[(i,j)] for i in range(len(supply)) for j in range(len(demand)))
pulp_lp_model.sense = plp.LpMinimize
pulp_lp_model.setObjective(objective)

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>5. Solve Problem</h2>
</div>

In [ ]:
status = pulp_lp_model.solve()

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>6. Results</h2>
</div>

In [ ]:
# This prints out the optimal value for all the decision variables
for i in range(len(supply)):
    for j in range(len(demand)):
        print("Optimal shipments from supply city %s to demand city %s is %d"%(list(supply.keys())[i], list(demand.keys())[j], quantity_arr[(i,j)].varValue))
# This gives the objective value that we were trying to maximize.
print("Total minimum shipping cost is %d"%pulp_lp_model.objective.value())

Optimal shipments from supply city A to demand city D is 0
Optimal shipments from supply city A to demand city E is 24
Optimal shipments from supply city A to demand city F is 0
Optimal shipments from supply city A to demand city G is 10
Optimal shipments from supply city A to demand city H is 0
Optimal shipments from supply city B to demand city D is 12
Optimal shipments from supply city B to demand city E is 0
Optimal shipments from supply city B to demand city F is 0
Optimal shipments from supply city B to demand city G is 32
Optimal shipments from supply city B to demand city H is 6
Optimal shipments from supply city C to demand city D is 18
Optimal shipments from supply city C to demand city E is 0
Optimal shipments from supply city C to demand city F is 12
Optimal shipments from supply city C to demand city G is 0
Optimal shipments from supply city C to demand city H is 0
Total minimum shipping cost is 1018


## SciPy Implementation

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>1. Constraints</h2>
</div>

**Order of Variables Considered - [AD,AE,AF,AG,AH,BD,BE,BF,BG,BH,CD,CE,CF,CG,CH]**

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(i) Supply Constraint</h4>
</div>

In [ ]:
supply_constraints_lhs = np.array([[1]*len(demand) + [0]*len(demand) + [0]*len(demand), \
                                   [0]*len(demand) + [1]*len(demand) + [0]*len(demand), 
                                   [0]*len(demand) + [0]*len(demand) + [1]*len(demand)])
supply_constraints_rhs = np.array(list(supply.values()))
print(supply_constraints_lhs, '\n', supply_constraints_rhs)

[[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]] 
 [70 50 30]


<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(ii) Demand Constraint</h4>
</div>

In [ ]:
demand_constraints_lhs = np.array([np.ndarray.flatten(np.array([-1]*len(supply)+ [0]*len(supply)*4).reshape(5,3).T), \
                                   np.ndarray.flatten(np.array([0]*len(supply) + [-1]*len(supply) + [0]*len(supply)*3).reshape(5,3).T), \
                                   np.ndarray.flatten(np.array([0]*len(supply)*2 + [-1]*len(supply) + [0]*len(supply)*2).reshape(5,3).T), \
                                   np.ndarray.flatten(np.array([0]*len(supply)*3 + [-1]*len(supply) + [0]*len(supply)*1).reshape(5,3).T), \
                                   np.ndarray.flatten(np.array([0]*len(supply)*4 + [-1]*len(supply)).reshape(5,3).T)])
demand_constraints_rhs = np.array([val*(-1) for val in list(demand.values())])
print(demand_constraints_lhs, '\n', demand_constraints_rhs)

[[-1  0  0  0  0 -1  0  0  0  0 -1  0  0  0  0]
 [ 0 -1  0  0  0  0 -1  0  0  0  0 -1  0  0  0]
 [ 0  0 -1  0  0  0  0 -1  0  0  0  0 -1  0  0]
 [ 0  0  0 -1  0  0  0  0 -1  0  0  0  0 -1  0]
 [ 0  0  0  0 -1  0  0  0  0 -1  0  0  0  0 -1]] 
 [-30 -24 -12 -42  -6]


<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(iii) Non-Negativity Constraint</h4>
</div>

In [ ]:
non_neg_constraints_lhs = np.diag(np.full(len(supply)*len(demand),-1))
non_neg_constraints_rhs = np.array([0]*len(supply)*len(demand))

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h4 style='margin:10px 5px'>(iv) All Constraints</h4>
</div>

In [ ]:
all_constraints_lhs = np.concatenate((supply_constraints_lhs,demand_constraints_lhs,non_neg_constraints_lhs),axis=0)
all_constraints_rhs = np.concatenate((supply_constraints_rhs, demand_constraints_rhs,non_neg_constraints_rhs))
print(all_constraints_lhs, '\n\n', all_constraints_rhs)

[[ 1  1  1  1  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  1  1  1  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  1  1  1  1]
 [-1  0  0  0  0 -1  0  0  0  0 -1  0  0  0  0]
 [ 0 -1  0  0  0  0 -1  0  0  0  0 -1  0  0  0]
 [ 0  0 -1  0  0  0  0 -1  0  0  0  0 -1  0  0]
 [ 0  0  0 -1  0  0  0  0 -1  0  0  0  0 -1  0]
 [ 0  0  0  0 -1  0  0  0  0 -1  0  0  0  0 -1]
 [-1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 -1

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>2. Objective Function</h2>
</div>

In [ ]:
obj = np.ndarray.flatten(cost_arr)
obj

array([16,  7, 17, 14, 19,  9, 11, 16, 10,  5, 10, 18,  6, 13,  8])

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>3. Solve Problem</h2>
</div>

In [ ]:
quantity_arr = linprog(obj, A_ub=all_constraints_lhs, b_ub=all_constraints_rhs, method="simplex")

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>4. Results</h2>
</div>

In [ ]:
# This prints out the optimal value for all the decision variables
for i in range(len(supply)):
    for j in range(len(demand)):
        print("Optimal shipments from supply city %s to demand city %s is %d"%(list(supply.keys())[i], list(demand.keys())[j], quantity_arr.x[len(demand)*i+j]))
# This gives the objective value that we were trying to maximize.
print("Total minimum shipping cost is %d"%quantity_arr.fun)

Optimal shipments from supply city A to demand city D is 0
Optimal shipments from supply city A to demand city E is 24
Optimal shipments from supply city A to demand city F is 0
Optimal shipments from supply city A to demand city G is 10
Optimal shipments from supply city A to demand city H is 0
Optimal shipments from supply city B to demand city D is 12
Optimal shipments from supply city B to demand city E is 0
Optimal shipments from supply city B to demand city F is 0
Optimal shipments from supply city B to demand city G is 32
Optimal shipments from supply city B to demand city H is 6
Optimal shipments from supply city C to demand city D is 18
Optimal shipments from supply city C to demand city E is 0
Optimal shipments from supply city C to demand city F is 12
Optimal shipments from supply city C to demand city G is 0
Optimal shipments from supply city C to demand city H is 0
Total minimum shipping cost is 1018


## Google OR Tools Implementation

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>1. Initializing Model</h2>
</div>

In [ ]:
#Storing Data
data = {}
data['constraint_coef'] = all_constraints_lhs
data['bounds'] = all_constraints_rhs
data['obj_coef'] = obj
data['num_var'] = len(supply)*len(demand)
data['num_constraint'] = len(supply) + len(demand) + len(supply)*len(demand)

ortools_lp_model = pywraplp.Solver.CreateSolver('SCIP')

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>2. Define Decision Variables</h2>
</div>

In [ ]:
infinity = ortools_lp_model.infinity()
quantity_arr = {}
for p in range(data['num_var']):
    quantity_arr[p] = ortools_lp_model.IntVar(0, infinity, 'quantity_arr[%i]' % p)
print('Number of variables =', ortools_lp_model.NumVariables())

Number of variables = 15


<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>3. Constraints</h2>
</div>

In [ ]:
for q in range(data['num_constraint']):
     constraint_expr = [data['constraint_coef'][q][r] * quantity_arr[r] for r in range(data['num_var'])]
     ortools_lp_model.Add(sum(constraint_expr) <= data['bounds'][q])
print('Number of constraints =', ortools_lp_model.NumConstraints())

Number of constraints = 23


<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>4. Objective Function</h2>
</div>

In [ ]:
obj_expr = [data['obj_coef'][s] * quantity_arr[s] for s in range(data['num_var'])]
ortools_lp_model.Minimize(ortools_lp_model.Sum(obj_expr))

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>5. Solve Problem</h2>
</div>

In [ ]:
status = ortools_lp_model.Solve()

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>6. Results</h2>
</div>

In [ ]:
# This prints out the optimal value for all the decision variables
for i in range(len(supply)):
    for j in range(len(demand)):
        print("Optimal shipments from supply city %s to demand city %s is %d"%(list(supply.keys())[i], list(demand.keys())[j], quantity_arr[len(demand)*i+j].solution_value()))
# This gives the objective value that we were trying to maximize.
print("Total minimum shipping cost is %d"%round(ortools_lp_model.Objective().Value()))

Optimal shipments from supply city A to demand city D is 0
Optimal shipments from supply city A to demand city E is 24
Optimal shipments from supply city A to demand city F is 0
Optimal shipments from supply city A to demand city G is 10
Optimal shipments from supply city A to demand city H is 0
Optimal shipments from supply city B to demand city D is 12
Optimal shipments from supply city B to demand city E is 0
Optimal shipments from supply city B to demand city F is 0
Optimal shipments from supply city B to demand city G is 32
Optimal shipments from supply city B to demand city H is 6
Optimal shipments from supply city C to demand city D is 18
Optimal shipments from supply city C to demand city E is 0
Optimal shipments from supply city C to demand city F is 12
Optimal shipments from supply city C to demand city G is 0
Optimal shipments from supply city C to demand city H is 0
Total minimum shipping cost is 1018
